In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import os
import seaborn as sns

from matplotlib.colors import ListedColormap
from pathlib import Path
from digiforest_analysis.utils import plotting

### Parameters

In [ ]:
VELOCITY_THR = 0.1
FILTERING_WINDOW = "1s"


In [ ]:
# Missions
missions = {
    "M1": {
        "path": "/media/matias/T71/datasets/2023-09-11-thesis-legged-forester-autonomy/2023-05-03-13-07-17-mission-m1"
    },
    "M2": {
        "path": "/media/matias/T71/datasets/2023-09-11-thesis-legged-forester-autonomy/2023-05-04-14-36-05-mission-m2"
    },
    "M3": {
        "path": "/media/matias/T71/datasets/2023-09-11-thesis-legged-forester-autonomy/2023-05-04-14-50-00-mission-m3"
    },
    "M4": {
        "path": "/media/matias/T71/datasets/2023-09-11-thesis-legged-forester-autonomy/2023-05-04-19-14-10-mission-m4"
    },
    "M5": {
        "path": "/media/matias/T71/datasets/2023-09-11-thesis-legged-forester-autonomy/2023-05-05-08-10-22-mission-m5"
    },
}

output_path = "/media/matias/T71/datasets/2023-09-11-thesis-legged-forester-autonomy/"

## Matplotlib config

In [ ]:
cm = 1 / 2.54
plot_width = 8.89 * cm
plot_height = 8 * cm

plt.rcParams["font.size"] = 8

n_colors = 10

### Load pose files

In [ ]:
from scipy.spatial.transform import Rotation as R


def read_poses_file(filename, base_inverted=False):
    df = pd.read_csv(filename)
    df = df.drop_duplicates()

    # Generate timestamp from sec and nsec
    ts = 1e9 * df["sec"] + df["nsec"]  # In nanoseconds
    df.index = pd.to_datetime(ts)
    df = df[~df.index.duplicated(keep="first")]

    # Parse data
    poses = {}
    for ts, x, y, z, qx, qy, qz, qw in zip(
        df.index, df["x"], df["y"], df["z"], df["qx"], df["qy"], df["qz"], df["qw"]
    ):
        poses[f"{ts:.10f}"] = np.eye(4)
        poses[f"{ts:.10f}"][0:3, 3] = np.array([x, y, z])
        poses[f"{ts:.10f}"][0:3, 0:3] = R.from_quat([qx, qy, qz, qw]).as_matrix()

        # TODO: fix base inversion

    return df, poses

### Load twist files

In [ ]:
def read_twist_file(filename, base_inverted=False):
    df = pd.read_csv(filename)
    df = df.drop_duplicates()

    # Generate timestamp from sec and nsec
    ts = 1e9 * df["sec"] + df["nsec"]  # In nanoseconds
    df.index = pd.to_datetime(ts)
    df = df[~df.index.duplicated(keep="first")]

    # Correct twist due to base inversion
    if BASE_INVERTED:
        df["vx"] *= -1
        df["vy"] *= -1

    # Speeds
    df["lin_speed"] = (df["vx"] ** 2 + df["vy"] ** 2).pow(1.0 / 2)
    df["ang_speed"] = df["wz"].abs()

    return df

### Load other operator's signals

In [ ]:
def read_param_change_file(filename, base_inverted=False):
    df = pd.read_csv(filename)
    df = df.drop_duplicates()

    # Generate timestamp from sec and nsec
    ts = 1e9 * df["sec"] + df["nsec"]  # In nanoseconds
    df.index = pd.to_datetime(ts)
    df = df[~df.index.duplicated(keep="first")]

    return df

### Read signals

In [ ]:
for k, m in missions.items():
    mission_path = m["path"]

    missions[k]["df_state_poses"], _ = read_poses_file(
        os.path.join(mission_path, "states/state_pose_data.csv"),
        base_inverted=BASE_INVERTED,
    )
    missions[k]["df_state_twist"] = read_twist_file(
        os.path.join(mission_path, "states/state_twist_data.csv"),
        base_inverted=BASE_INVERTED,
    )
    missions[k]["df_reference_twist"] = read_twist_file(
        os.path.join(mission_path, "states/reference_twist_data.csv"),
        base_inverted=BASE_INVERTED,
    )
    missions[k]["df_operator_twist"] = read_twist_file(
        os.path.join(mission_path, "states/operator_twist_data.csv"),
        base_inverted=BASE_INVERTED,
    )
    missions[k]["df_local_planner_param"] = read_param_change_file(
        os.path.join(mission_path, "states/local_planner_param_data.csv"),
    )
    missions[k]["df_rmp_param"] = read_param_change_file(
        os.path.join(mission_path, "states/rmp_param_data.csv"),
    )

### Align time series

In [ ]:
for k, m in missions.items():
    # Extend indices
    joined_indices = (
        missions[k]["df_state_twist"]
        .index.union(missions[k]["df_reference_twist"].index)
        .drop_duplicates()
    )
    joined_indices = joined_indices.union(
        missions[k]["df_operator_twist"].index
    ).drop_duplicates()
    joined_indices = joined_indices.union(
        missions[k]["df_local_planner_param"].index
    ).drop_duplicates()
    joined_indices = joined_indices.union(
        missions[k]["df_rmp_param"].index
    ).drop_duplicates()

    missions[k]["df_state_twist"] = missions[k]["df_state_twist"].reindex(
        index=joined_indices
    )
    missions[k]["df_state_twist"] = missions[k]["df_state_twist"].interpolate(
        method="index"
    )

    missions[k]["df_operator_twist"] = missions[k]["df_operator_twist"].reindex(
        index=joined_indices
    )
    missions[k]["df_operator_twist"] = missions[k]["df_operator_twist"].interpolate(
        method="index"
    )

    missions[k]["df_reference_twist"] = missions[k]["df_reference_twist"].reindex(
        index=joined_indices
    )
    missions[k]["df_reference_twist"] = missions[k]["df_reference_twist"].interpolate(
        method="index"
    )

    missions[k]["df_local_planner_param"] = missions[k][
        "df_local_planner_param"
    ].reindex(index=joined_indices)
    missions[k]["df_rmp_param"] = missions[k]["df_rmp_param"].reindex(
        index=joined_indices
    )

## Estimate metrics

### Operator interventions
This is required for autonomy metrics

In [ ]:
for k, m in missions.items():
    # Interventions from safety officer
    missions[k]["df_state_twist"]["safety_intervention"] = (
        missions[k]["df_operator_twist"]["lin_speed"] > VELOCITY_THR
    )
    missions[k]["df_state_twist"]["safety_intervention"] = missions[k][
        "df_state_twist"
    ]["safety_intervention"].interpolate(method="pad")

    # Interventions from forestry operator
    missions[k]["df_state_twist"]["operator_intervention"] = (
        ~missions[k]["df_local_planner_param"]["sec"].isna()
    ) + (~missions[k]["df_rmp_param"]["sec"].isna())

### Normalize time

In [ ]:
for k, m in missions.items():
    ref_ts = missions[k]["df_state_twist"].index[0]

    missions[k]["df_state_poses"].index = missions[k]["df_state_poses"].index - ref_ts
    missions[k]["df_state_twist"].index = missions[k]["df_state_twist"].index - ref_ts
    missions[k]["df_reference_twist"].index = missions[k]["df_reference_twist"].index - ref_ts
    missions[k]["df_operator_twist"].index = missions[k]["df_operator_twist"].index - ref_ts

## Plots

### Autonomy

In [ ]:
from scipy import signal

plot_width = 50 * cm
plot_height = 20 * cm

N = len(missions)

fig, ax = plt.subplots(
    N,
    1,
    figsize=(plot_width, plot_height),
    constrained_layout=False,
    dpi=300,
    sharex=True,
)

for i, (k,v) in enumerate(missions.items()):
    ax[i].plot(missions[k]["df_state_twist"].index.total_seconds(), missions[k]["df_state_twist"]["safety_intervention"], linewidth=1)
    # Filter signal
    missions[k]["df_state_twist"]["safety_intervention_filt"] = missions[k]["df_state_twist"]["safety_intervention"] * 0 # Just initial value
    missions[k]["df_state_twist"]["safety_intervention_filt"] = missions[k]["df_state_twist"]["safety_intervention"].rolling(FILTERING_WINDOW, center=True).median() > 0.5
    ax[i].plot(missions[k]["df_state_twist"].index.total_seconds(), missions[k]["df_state_twist"]["safety_intervention_filt"], linewidth=0.8)

    # We find the peaks in the original interventions signal, that should correspond to the center
    peaks, properties = signal.find_peaks(missions[k]["df_state_twist"]["safety_intervention_filt"],  distance=1)
    missions[k]["safety_intervention_num"] = len(peaks)
    ax[i].plot(missions[k]["df_state_twist"].index[peaks].total_seconds(), missions[k]["df_state_twist"]["operator_intervention"][peaks], ".", linewidth=0.1)
    
    # We find the left and right edges of the peak
    results_half = signal.peak_widths(missions[k]["df_state_twist"]["safety_intervention_filt"], peaks, rel_height=0.5)
    pos_l = results_half[2].astype(int)
    pos_r = results_half[3].astype(int)

    t = missions[k]["df_state_twist"].index.total_seconds()

    # Intervention time
    missions[k]["interventions"] = []
    for l,r in zip(pos_l, pos_r):
        dt = t[r] - t[l]
        missions[k]["interventions"].append(dt)
    
    # Autonomy time
    # We need to do some hacking here
    if len(pos_l) and len(pos_r):
        pos_r = np.insert(pos_r, 0, 0, axis=0)
        pos_l = np.insert(pos_l, -1, len(t)-1, axis=0)

    missions[k]["autonomy"] = []
    for l,r in zip(pos_l, pos_r):
        dt = t[l] - t[r]
        if dt > 1:
            missions[k]["autonomy"].append(dt)


print("Number of interventions")
for k,v in missions.items():
    print(f"{k}: {v['safety_intervention_num']}")

import scipy.integrate as integrate
print("")
print("Autonomy percentage")
for k,v in missions.items():
    intervention_seconds = integrate.simpson(
        missions[k]["df_state_twist"]["safety_intervention_filt"], missions[k]["df_state_twist"]["safety_intervention_filt"].index.total_seconds()
    ).item()

    mission_time = df_state_twist.index.total_seconds()[-1]

    print(f"{k}: {100*(1.0 - intervention_seconds / mission_time):.2f}")

### Interventions plot

In [ ]:
accumulated_interventions = []
list_interventions = []
color_interventions = []
label_interventions = []
for i, (k,m) in enumerate(missions.items()):
    accumulated_interventions.extend(m["interventions"])
    list_interventions.append(np.asarray(m["interventions"], dtype=int))
    color_interventions.append(plotting.color_palette[i])
    label_interventions.append(k)
accumulated_interventions=np.asarray(accumulated_interventions, dtype=int)

# Plotting settings
plot_width = 12 * cm
plot_height = 6 * cm

# Plot
fig, ax = plt.subplots(
    1,
    1,
    figsize=(plot_width, plot_height),
    constrained_layout=False,
    dpi=300,
    sharex=True,
)
# Axes
ax.set_axisbelow(True)
# ax.set_aspect("equal")
ax.grid(which="major", color=plotting.gray_palette_str["20"], linewidth=0.7)
ax.grid(
    which="minor", color=plotting.gray_palette_str["10"], linestyle=":", linewidth=0.5
)
ax.minorticks_on()
# Plot
ax.hist(list_interventions, bins=20, cumulative=False, density=False, stacked=True, color=color_interventions, label=label_interventions, rwidth=0.9)
# ax.set_title("Distribution of intervention time")
ax.set_xlabel("Intervention duration [s]")
ax.set_ylabel("Frequency")

lgnd = ax.legend(edgecolor=(1, 1, 1, 0), framealpha=0.9, ncol=1)
for handle in lgnd.legend_handles:
    try:
        handle.set_sizes([40.0])
    except Exception as e:
        print(e)

# fig.set_tight_layout(True)
fig.savefig(os.path.join(output_path, "intervention_distribution.pdf"))


### Autonomy plot

In [ ]:
accumulated_autonomy = []
list_autonomy = []
color_autonomy = []
label_autonomy = []
for i, (k,m) in enumerate(missions.items()):
    accumulated_autonomy.extend(m["autonomy"])
    list_autonomy.append(np.asarray(m["autonomy"]))
    color_autonomy.append(plotting.color_palette[i])
accumulated_autonomy=np.asarray(accumulated_autonomy)

# Plot
fig, ax = plt.subplots(
    1,
    1,
    figsize=(plot_width, plot_height),
    constrained_layout=False,
    dpi=300,
    sharex=True,
)
# Axes
ax.set_axisbelow(True)
# ax.set_aspect("equal")
ax.grid(which="major", color=plotting.gray_palette_str["20"], linewidth=0.7)
ax.grid(
    which="minor", color=plotting.gray_palette_str["10"], linestyle=":", linewidth=0.5
)
ax.minorticks_on()
# Plot
ax.hist(all_autonomy, bins=25, cumulative=False, density=False, color=plotting.color_palette_str["blue"])
ax.set_title("Distribution of autonomous operation time")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Frequency")

fig.set_tight_layout(True)
fig.savefig(os.path.join(output_path, "autonomous_time.pdf"))

### Robot velocity

In [ ]:
smoothing_window = "3000ms"
linewidth = 0.75

plot_width = 15 * cm
plot_height = 4 * cm

N = len(missions)


for i, (k, m) in enumerate(missions.items()):
    fig, ax = plt.subplots(
        1,
        1,
        figsize=(plot_width, plot_height),
        constrained_layout=False,
        dpi=300,
        sharex=True,
    )

    # Axes
    ax.set_axisbelow(True)
    # ax.set_aspect("equal")
    ax.grid(which="major", color=plotting.gray_palette_str["20"], linewidth=0.7)
    ax.grid(
        which="minor", color=plotting.gray_palette_str["10"], linestyle=":", linewidth=0.5
    )
    ax.minorticks_on()

    ax.plot(
        missions[k]["df_state_twist"].index.total_seconds(),
        missions[k]["df_state_twist"]["lin_speed"].rolling(smoothing_window, center=True).mean(),
        label="Robot speed",
        linewidth=linewidth,
        color=plotting.color_palette_str["blue"],
    )
    ax.plot(
        missions[k]["df_reference_twist"].index.total_seconds(),
        missions[k]["df_reference_twist"]["lin_speed"].rolling(smoothing_window, center=True).mean(),
        label="Local planner reference",
        linewidth=linewidth,
        color=plotting.color_palette_str["orange"],
    )

    # if missions[k]["df_state_twist"]["operator_intervention"].any():
    #     ax[i].fill_between(
    #         missions[k]["df_state_twist"]["operator_intervention"].index.total_seconds(),
    #         missions[k]["df_state_twist"]["operator_intervention"].index.total_seconds() * 0,
    #         missions[k]["df_state_twist"]["operator_intervention"],
    #         label="Operator action",
    #         linewidth=1,
    #         linestyle="-",
    #         color=plotting.gray_palette_str["90"],
    #         alpha=1.0,
    #     )

    if missions[k]["df_state_twist"]["safety_intervention_filt"].any():
        ax.fill_between(
            missions[k]["df_state_twist"]["safety_intervention_filt"].index.total_seconds(),
            missions[k]["df_state_twist"]["safety_intervention_filt"].index.total_seconds() * 0,
            missions[k]["df_state_twist"]["safety_intervention_filt"],
            label="Safety intervention",
            linewidth=0,
            color=plotting.gray_palette_str["60"],
            alpha=0.7,
        )

    # Legends
    lgnd = ax.legend(edgecolor=(1, 1, 1, 0), framealpha=0.9, loc=(0, 1.03), ncol=3)
    for handle in lgnd.legend_handles:
        try:
            handle.set_sizes([40.0])
        except Exception as e:
            print(e)

    for handle in lgnd.get_lines():
        handle.set_linewidth(1)

    # ax.set_title('Computation Time')
    ax.margins(x=0, y=0)
    ax.set_ylabel("Speed [m/s]")
    ax.set_xlabel("Time [s]")

    # Export
    fig.set_tight_layout(True)
    fig.savefig(os.path.join(output_path, f"mission_velocity_{k.lower()}.pdf"))